## <center> Análise de Cluster <center>

In [1]:
import pandas as pd
import sqlalchemy
import mysql.connector
import seaborn as sns
import numpy as np
import datetime as dt
import xlrd

from sqlalchemy import create_engine

import time
from time import gmtime
from time import strftime

from openpyxl import Workbook
from openpyxl import Workbook, load_workbook
from pandas import ExcelWriter

import plotly.express as px


from ortools.sat.python import cp_model


from datetime import datetime
from datetime import timedelta
from datetime import date


from ortools.sat.python import cp_model
import math
from pathlib import Path, PureWindowsPath

pd.set_option("display.float_format", lambda x: "%.2f" %x)

import pandas as pd
from glob import glob



* ### Planilha Cupom Médio das Lojas

Cupom Médio = Total vendido por dia/ Qtd de cupons por dia

In [2]:
planilha = "C:/Users/anunes/R-DIAS ASSESSORIA PARA O VAREJO/R-Lab - Cluster/CupomMedio_Lojas/CupomMedio_LojasNicolini.xlsx"
CupomMedioLojas = pd.read_excel(planilha)

In [3]:
CupomMedioLojas = CupomMedioLojas.rename(columns={'NroCupom': 'QtdCupons_Dia', 'total':'total_Dia'})
CupomMedioLojas['Mes_Ano'] = CupomMedioLojas['Data'].dt.strftime('%m-%Y')

In [4]:
CupomMedioLojas

,Loja,Data,total_Dia,QtdCupons_Dia,CupomMedio,Mes_Ano
0,1,2020-12-14,157997.71,1106,142.86,12-2020
1,1,2020-12-15,149245.69,1255,118.92,12-2020
2,1,2020-12-16,137492.32,1152,119.35,12-2020
3,1,2020-12-17,134912.09,1377,97.98,12-2020
4,1,2020-12-18,201943.76,1402,144.04,12-2020
...,...,...,...,...,...,...
3588,9,2021-12-23,210513.73,2385,88.27,12-2021
3589,9,2021-12-24,201137.69,2371,84.83,12-2021
3590,9,2021-12-26,32924.74,569,57.86,12-2021
3591,9,2021-12-27,68565.11,1236,55.47,12-2021


In [5]:
Total_QtdCupons_Mes = CupomMedioLojas.groupby(['Mes_Ano']).agg({'total_Dia': np.sum, 'QtdCupons_Dia': np.sum}).reset_index()

In [6]:
Total_QtdCupons_Mes.to_excel("Total_QtdCupons_Mes.xlsx")

In [20]:
## Adiçao da data e hora

CupomMedioLojas['Dia_da_Semana'] = CupomMedioLojas['Data'].dt.day_name().replace(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 
                                                                                 'Friday', 'Saturday','Sunday'],
                                                                                ['seg', 'ter', 'qua', 'qui', 'sex', 'sab', 'dom'])

CupomMedioLojas.loc[: ,'Horario_Abertura'] = '08:00:00'
    
for i in range(len(CupomMedioLojas)):
    nroloja = CupomMedioLojas.loc[i,"Loja"]

    if (nroloja == 2) | (nroloja == 6) | (nroloja == 7):
        CupomMedioLojas.loc[i ,'Horario_Fechamento'] = np.where(CupomMedioLojas.loc[i, 'Dia_da_Semana'] == 'dom', "21:00:00", "21:30:00")

    elif (nroloja == 1):
        CupomMedioLojas.loc[i , 'Horario_Fechamento'] = np.where(CupomMedioLojas.loc[i, 'Dia_da_Semana'] == 'dom', "21:30:00", "22:00:00")


    elif (nroloja == 3):
        CupomMedioLojas.loc[i, 'Horario_Fechamento'] = np.where(CupomMedioLojas.loc[i, 'Dia_da_Semana'] != 'dom', '22:00:00', "21:00:00")  

    elif (nroloja == 4) | (nroloja == 5): 
        CupomMedioLojas.loc[i, 'Horario_Fechamento'] = np.where(CupomMedioLojas.loc[i, 'Dia_da_Semana'] != 'dom', '21:30:00', "13:00:00")
    
    elif (nroloja == 9) | (nroloja == 10): 
        CupomMedioLojas.loc[i, 'Horario_Fechamento'] = np.where(CupomMedioLojas.loc[i, 'Dia_da_Semana'] != 'dom', '21:00:00', "13:00:00")


    elif (nroloja == 8):
        CupomMedioLojas.loc[i, 'Horario_Fechamento'] = np.where(CupomMedioLojas.loc[i, 'Dia_da_Semana'] != 'dom', '21:00:00', 0)


In [30]:
CupomMedioLojas.loc[:, 'Horario_MeioDia'] = '12:00:00'
CupomMedioLojas.loc[:, 'Horario_6hsTarde'] = '18:00:00'

* ### Planilha Média de Itens por Cupom

In [ ]:
MediaItensCupom_Lojas = sorted(glob(r'C:/Users/anunes/R-DIAS ASSESSORIA PARA O VAREJO/R-Lab - Cluster/ItensCupom_Lojas/*.xlsx'))
MediaItensCupom_Lojas 

In [ ]:
MediaItensCupom_Lojas = pd.concat((pd.read_excel(cont) for cont in MediaItensCupom_Lojas))
MediaItensCupom_Lojas

Exemplo: No dia 2021-02-03, o cupom 185906 teve a média de 4 itens.

In [ ]:
QtdPdvs = pd.DataFrame(data = {'loja': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],'Pdvs':  [18, 10, 10, 7, 9, 10, 14, 15, 12, 12]})